<a href="https://colab.research.google.com/github/konstantins7/rabota1/blob/main/__200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-auth requests pandas google-api-python-client

import requests
import pandas as pd
import datetime
import os
import time
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from googleapiclient.discovery import build

# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Путь к файлам на Google Drive
KEY_FILE_LOCATION = '/content/drive/MyDrive/Colab Notebooks/medvitrina24kz-348e2b42e091.json'  # путь к загруженному JSON файлу
TXT_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/urlmedvitrina24.txt'  # путь к загруженному TXT файлу
PROGRESS_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/indexing_progress.csv'  # файл для отслеживания прогресса

SCOPES = ["https://www.googleapis.com/auth/indexing"]
INDEXING_URL = "https://indexing.googleapis.com/v3/urlNotifications:publish"
DAILY_QUOTA = 200  # Пример дневной квоты, замените на ваше значение

# Функция для логирования сообщений
def log_message(message):
    with open("/content/drive/MyDrive/Colab Notebooks/indexing_log.txt", "a") as log_file:
        log_file.write(f"{datetime.datetime.now()}: {message}\n")

# Проверка наличия файлов
def check_file_exists(file_path):
    if not os.path.exists(file_path):
        log_message(f"Ошибка: Файл {file_path} не найден.")
        print(f"Ошибка: Файл {file_path} не найден.")
        return False
    log_message(f"Файл {file_path} найден.")
    print(f"Файл {file_path} найден.")
    return True

# Проверка всех необходимых файлов
if not check_file_exists(KEY_FILE_LOCATION):
    log_message("Скрипт завершен из-за отсутствия файла ключей.")
    exit(1)
if not check_file_exists(TXT_FILE_PATH):
    log_message("Скрипт завершен из-за отсутствия файла с URL.")
    exit(1)

if not os.path.exists(PROGRESS_FILE_PATH):
    log_message(f"Файл {PROGRESS_FILE_PATH} не найден, будет создан новый файл.")

# Создание учетных данных и авторизованного сеанса
try:
    credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION, scopes=SCOPES)
    authed_session = AuthorizedSession(credentials)
    log_message("Учетные данные успешно созданы и сессия авторизована.")
except Exception as e:
    log_message(f"Ошибка при создании учетных данных: {e}")
    raise

# Загрузка данных из TXT файла
try:
    with open(TXT_FILE_PATH, 'r') as file:
        urls = [line.strip() for line in file]
    log_message(f"Файл {TXT_FILE_PATH} успешно прочитан.")
except Exception as e:
    log_message(f"Ошибка при чтении файла {TXT_FILE_PATH}: {e}")
    raise

# Создание или загрузка файла прогресса
try:
    progress = pd.read_csv(PROGRESS_FILE_PATH)
    log_message(f"Файл прогресса {PROGRESS_FILE_PATH} успешно загружен.")
except FileNotFoundError:
    progress = pd.DataFrame(urls, columns=['Product URL'])
    progress['Indexed'] = False
    progress.to_csv(PROGRESS_FILE_PATH, index=False)
    log_message(f"Создан новый файл прогресса: {PROGRESS_FILE_PATH}")

# Функция для отправки URL-адреса на индексацию
def publish_url(url):
    data = {
        "url": url,
        "type": "URL_UPDATED"
    }
    for attempt in range(5):  # Повторные попытки до 5 раз
        response = authed_session.post(INDEXING_URL, json=data)
        log_message(f"Попытка {attempt + 1} публикации URL {url}: {response.status_code}, {response.text}")
        if response.status_code == 503:
            time.sleep(2 ** attempt)  # Экспоненциальное ожидание перед повторной попыткой
        else:
            return response
    return response  # Возвращаем последний ответ

# Проверка квоты перед началом индексации
def check_quota():
    try:
        service = build('indexing', 'v3', credentials=credentials)
        response = service.urlNotifications().getMetadata(url="https://med.vitrina24.kz").execute()
        log_message(f"Информация о квоте: {response}")
        return response
    except Exception as e:
        log_message(f"Ошибка при проверке квоты: {e}")
        return None

# Определение текущей даты
today = datetime.date.today()

# Отслеживание количества отправленных URL
urls_sent_today = 0

# Проверка квоты перед началом индексации
quota_info = check_quota()
if quota_info:
    log_message(f"Информация о квоте: {quota_info}")
    remaining_quota = DAILY_QUOTA - urls_sent_today
    log_message(f"Оставшаяся квота: {remaining_quota}")
else:
    log_message("Не удалось получить информацию о квоте.")
    remaining_quota = 0

if remaining_quota > 0:
    # Обход по URL-адресам и отправка на индексацию
    for index, row in progress.iterrows():
        if urls_sent_today >= remaining_quota:
            log_message("Достигнута дневная квота. Попробуйте снова завтра.")
            print("Достигнута дневная квота. Попробуйте снова завтра.")
            break

        if not row['Indexed']:
            url = row['Product URL']
            response = publish_url(url)

            if response.status_code == 200:
                progress.at[index, 'Indexed'] = True
                urls_sent_today += 1
                log_message(f"URL отправлен успешно: {url}")
            elif response.status_code == 429:
                log_message("Достигнута дневная квота. Попробуйте снова завтра.")
                print("Достигнута дневная квота. Попробуйте снова завтра.")
                break
            elif response.status_code == 403:
                log_message(f"Ошибка при отправке {url}: {response.status_code} {response.text}")
                print(f"Ошибка при отправке {url}: {response.status_code} {response.text}")
                break
            else:
                log_message(f"Ошибка при отправке {url}: {response.status_code} {response.text}")
                print(f"Ошибка при отправке {url}: {response.status_code} {response.text}")

    # Сохранение обновленного файла прогресса после каждой итерации
    progress.to_csv(PROGRESS_FILE_PATH, index=False)

else:
    log_message("Достигнута дневная квота или произошла ошибка при проверке квоты.")
    print("Достигнута дневная квота или произошла ошибка при проверке квоты.")

# Сохранение обновленного файла прогресса в конце выполнения
progress.to_csv(PROGRESS_FILE_PATH, index=False)

log_message(f"Сегодня было отправлено {urls_sent_today} URL-адресов.")
print(f"Сегодня было отправлено {urls_sent_today} URL-адресов.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Файл /content/drive/MyDrive/Colab Notebooks/medvitrina24kz-348e2b42e091.json найден.
Файл /content/drive/MyDrive/Colab Notebooks/urlmedvitrina24.txt найден.
Ошибка при отправке https://med.vitrina24.kz/apteki/medikamenty/maksigrippin-d-d-10-ship/: 503 {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}

Достигнута дневная квота. Попробуйте снова завтра.
Сегодня было отправлено 198 URL-адресов.
